In [2]:
!pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9657 sha256=475d03aa448f7e910ccf90a1eec6aafea9dc024ada41335801183d2f0a0bab04
  Stored in directory: /Users/zayesosa/Library/Caches/pip/wheels/04/5f/3e/46cc37c5d698415694d83f607f833f83f0149e49b3af9d0f38
Successfully built wget


In [4]:
!wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/car_fuel_efficiency.csv

--2025-10-05 20:15:31--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/car_fuel_efficiency.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 874188 (854K) [text/plain]
Saving to: 'car_fuel_efficiency.csv.1'

car_fuel_efficiency 100%[===================>] 853.70K  1.25MB/s    in 0.7s    

2025-10-05 20:15:32 (1.25 MB/s) - 'car_fuel_efficiency.csv.1' saved [874188/874188]



In [42]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.metrics import mean_squared_error

In [43]:
cfe=pd.read_csv("car_fuel_efficiency.csv")

In [44]:
cfe.head()


,engine_displacement,num_cylinders,horsepower,vehicle_weight,acceleration,model_year,origin,fuel_type,drivetrain,num_doors,fuel_efficiency_mpg
0,170,3.0,159.0,3413.433759,17.7,2003,Europe,Gasoline,All-wheel drive,0.0,13.231729
1,130,5.0,97.0,3149.664934,17.8,2007,USA,Gasoline,Front-wheel drive,0.0,13.688217
2,170,NaN,78.0,3079.038997,15.1,2018,Europe,Gasoline,Front-wheel drive,0.0,14.246341
3,220,4.0,NaN,2542.392402,20.2,2009,USA,Diesel,All-wheel drive,2.0,16.912736
4,210,1.0,140.0,3460.870990,14.4,2009,Europe,Gasoline,All-wheel drive,2.0,12.488369


In [45]:
##Question 1
df=cfe[['engine_displacement','horsepower','vehicle_weight','model_year','fuel_efficiency_mpg']]
missing_values= df.isna().any()
print(missing_values)

engine_displacement    False
horsepower              True
vehicle_weight         False
model_year             False
fuel_efficiency_mpg    False
dtype: bool


In [46]:
df['horsepower'].median()

149.0

In [47]:
n = len(df)
np.random.seed(42)
idx = np.arange(n)
np.random.shuffle(idx)

df_shuffled = df.iloc[idx].reset_index(drop=True)

In [48]:
df_shuffled.head()

,engine_displacement,horsepower,vehicle_weight,model_year,fuel_efficiency_mpg
0,220,144.0,2535.887591,2009,16.642943
1,160,141.0,2741.170484,2019,16.298377
2,230,155.0,2471.880237,2017,18.591822
3,150,206.0,3748.164469,2015,11.818843
4,300,111.0,2135.716359,2006,19.402209


In [49]:
n_train = int(n * 0.6)
n_val = int(n * 0.2)
n_test = n - n_train - n_val

In [50]:
df_train=df_shuffled.iloc[:n_train].copy()
df_val=df_shuffled.iloc[n_train:n_train + n_val].copy()
df_test=df_shuffled.iloc[n_train+ n_val:].copy()

In [51]:
print(len(df_train))
print(len(df_val))
print(len(df_test))

5822
1940
1942


In [52]:
# Question 3
missing_column='horsepower'
target_column='fuel_efficiency_mpg'

# Select only numeric columns for features (excluding target)
numeric_columns = df.select_dtypes(include=[np.number]).columns.tolist()
feature_columns = [col for col in numeric_columns if col != target_column]

print(f"\nFeature columns: {feature_columns}")
print(f"Missing column to handle: {missing_column}")


Feature columns: ['engine_displacement', 'horsepower', 'vehicle_weight', 'model_year']
Missing column to handle: horsepower


In [53]:
# Function to train and evaluate model
def train_and_evaluate(df_train, df_val, fill_value=None, use_mean=False):
    # Make copies
    train = df_train.copy()
    val = df_val.copy()
    
    # Handle missing values
    if use_mean:
        # Compute mean from training set only
        mean_value = train[missing_column].mean()
        train[missing_column].fillna(mean_value, inplace=True)
        val[missing_column].fillna(mean_value, inplace=True)
        print(f"Filled missing values with mean: {mean_value:.4f}")
    else:
        # Fill with specified value (0)
        train[missing_column].fillna(fill_value, inplace=True)
        val[missing_column].fillna(fill_value, inplace=True)
        print(f"Filled missing values with: {fill_value}")
    
    # Prepare X and y
    X_train = train[feature_columns].values
    y_train = train[target_column].values
    
    X_val = val[feature_columns].values
    y_val = val[target_column].values
    
    # Train linear regression model (no regularization)
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # Predict on validation set
    y_pred = model.predict(X_val)
    
    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    
    return rmse

In [54]:
## when filling with zero
rmse_zero = train_and_evaluate(df_train, df_val, fill_value=0, use_mean=False)
rmse_zero = round(rmse_zero, 2)
print(f"RMSE with 0: {rmse_zero}")

Filled missing values with: 0
RMSE with 0: 0.52


In [55]:
## when filling with mean
rmse_mean = train_and_evaluate(df_train, df_val, fill_value=None, use_mean=True)
rmse_mean = round(rmse_mean, 2)
print(f"RMSE with mean: {rmse_mean}")

Filled missing values with mean: 149.5435
RMSE with mean: 0.46


In [56]:
##Mean with missing values is more accurate

In [57]:
# Q4: regularized linear (fill NA with 0) test r in [0,0.01,0.1,1,5,10,100]
r_list = [0, 0.01, 0.1, 1, 5, 10, 100]
results_q4 = {}
# prepare data filled with zero (train mean for r doesn't matter)
X_train = df_train[['engine_displacement','horsepower','vehicle_weight','model_year']].fillna(0).to_numpy()
y_train = df_train['fuel_efficiency_mpg'].to_numpy()
X_val = df_val[['engine_displacement','horsepower','vehicle_weight','model_year']].fillna(0).to_numpy()
y_val = df_val['fuel_efficiency_mpg'].to_numpy()
for r in r_list:
    # sklearn Ridge uses alpha parameter (alpha = r)
    # Note: r=0 can be LinearRegression
    if r == 0:
        model = LinearRegression()
    else:
        model = Ridge(alpha=r)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    results_q4[r] = round(rmse, 2)
print("\nQ4 results (r -> RMSE):")
for r, rm in results_q4.items():
    print(r, "->", rm)
best_r = min(results_q4, key=lambda k: (results_q4[k], k))  # pick smallest r if tie
print("Best r:", best_r)


Q4 results (r -> RMSE):
0 -> 0.52
0.01 -> 0.52
0.1 -> 0.52
1 -> 0.52
5 -> 0.52
10 -> 0.52
100 -> 0.52
Best r: 0


In [62]:
##other option
def train_linear_regression_reg(X, y, r=0.0):
    """
    Train regularized linear regression using the normal equation:
    w = (X^T X + r*I)^(-1) X^T y
    
    Where r is the regularization parameter
    """
    ones = np.ones(X.shape[0])
    X_with_bias = np.column_stack([ones, X])
    
    XTX = X_with_bias.T.dot(X_with_bias)
    reg = r * np.eye(XTX.shape[0])
    XTX = XTX + reg
    
    XTX_inv = np.linalg.inv(XTX)
    w = XTX_inv.dot(X_with_bias.T).dot(y)
    
    return w[0], w[1:]

def rmse(y_true, y_pred):
    """Calculate Root Mean Squared Error"""
    se = (y_true - y_pred) ** 2
    mse = se.mean()
    return np.sqrt(mse)

In [63]:
# Test different values of r
r_values = [0, 0.01, 0.1, 1, 5, 10, 100]
results = []

for r in r_values:
    # Train model
    w0, w = train_linear_regression_reg(X_train, y_train, r=r)
    
    # Predict on validation set
    y_pred = w0 + X_val.dot(w)
    
    # Calculate RMSE
    score = rmse(y_val, y_pred)
    score_rounded = round(score, 2)
    
    results.append({
        'r': r,
        'rmse': score_rounded
    })
    
    print(f"r = {r:6.2f} -> RMSE = {score_rounded:.2f}")

r =   0.00 -> RMSE = 0.52
r =   0.01 -> RMSE = 0.52
r =   0.10 -> RMSE = 0.52
r =   1.00 -> RMSE = 0.52
r =   5.00 -> RMSE = 0.52
r =  10.00 -> RMSE = 0.52
r = 100.00 -> RMSE = 0.52


In [70]:
# Q5: seeds 0..9 with fill 0, no regularization
# Prepare default splits with seed 42
def prepare_splits(df, seed):
    df_shuffled = df.sample(frac=1, random_state=seed).reset_index(drop=True)
    n = len(df_shuffled)
    n_train = int(0.6 * n)
    n_val = int(0.2 * n)
    df_train = df_shuffled.iloc[:n_train].reset_index(drop=True)
    df_val = df_shuffled.iloc[n_train:n_train+n_val].reset_index(drop=True)
    df_test = df_shuffled.iloc[n_train+n_val:].reset_index(drop=True)
    return df_train, df_val, df_test
df_train, df_val, df_test = prepare_splits(df, seed=42)

def linear_rmse(df_train, df_val, fill_with='zero'):
    # fill strategy for horsepower (the only NA)
    X_train = df_train[['engine_displacement','horsepower','vehicle_weight','model_year']].copy()
    X_val = df_val[['engine_displacement','horsepower','vehicle_weight','model_year']].copy()
    y_train = df_train['fuel_efficiency_mpg'].values
    y_val = df_val['fuel_efficiency_mpg'].values
    
    if fill_with == 'zero':
        X_train = X_train.fillna(0)
        X_val = X_val.fillna(0)
    elif fill_with == 'mean':
        # compute mean from training only
        mean_hp = X_train['horsepower'].mean()
        X_train['horsepower'] = X_train['horsepower'].fillna(mean_hp)
        X_val['horsepower'] = X_val['horsepower'].fillna(mean_hp)
        X_train = X_train.fillna(0)  # in case other cols have NA (they don't)
        X_val = X_val.fillna(0)
    else:
        raise ValueError("Unknown fill")
    # convert to numpy
    X_train = X_train.to_numpy()
    X_val = X_val.to_numpy()

    lr = LinearRegression()
    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    return rmse
    
seeds = list(range(10))
rmse_list = []
for s in seeds:
    tr, va, te = prepare_splits(df, seed=s)
    rm = linear_rmse(tr, va, fill_with='zero')
    rmse_list.append(rm)
std_rmse = np.std(rmse_list)
print("\nQ5 RMSE list:", [round(x,3) for x in rmse_list])
print("Q5 std (np.std):", round(std_rmse, 3))


Q5 RMSE list: [0.521, 0.522, 0.523, 0.516, 0.511, 0.529, 0.532, 0.51, 0.515, 0.513]
Q5 std (np.std): 0.007


In [78]:
train9, val9, test9 = prepare_splits(df, seed=9)
df_trainval=pd.concat([train9,val9], ignore_index=True)
X_trainval = df_trainval[['engine_displacement','horsepower','vehicle_weight','model_year']].fillna(0).to_numpy()
y_trainval = df_trainval['fuel_efficiency_mpg'].to_numpy()
X_test = test9[['engine_displacement','horsepower','vehicle_weight','model_year']].fillna(0).to_numpy()
y_test = test9['fuel_efficiency_mpg'].to_numpy()

In [79]:
model = Ridge(alpha=0.001)
model.fit(X_trainval, y_trainval)
y_pred_test = model.predict(X_test)
rmse_test = mean_squared_error(y_test, y_pred_test, squared=False)
print("\nQ6 RMSE on test (r=0.001):", round(rmse_test, 3))


Q6 RMSE on test (r=0.001): 0.515
